# Étude à l'échelle de la consommation par adresse

Nous tâchons désormais de savoir s'il est possible pour un producteur d'électricité de prévoir la consommation d'électricité d'une résidence à partir d'un autre type de données plus local que des données départementales : des données à l'échelle des communes et des adresses. Nous continuons à considérer la météo comme une variable potentiellement pertinente pour comprendre la consommation, et nous cherchons à ajouter à cela la performance énergétique des bâtiments en passant par l'indice de DPE (Diagnostic de performance énergétique). Ainsi, on cherche à savoir si les producteurs d'électricité, connaissant la notation énergétique d'un bâtiment toutes énergies comprises peuvent en prédire efficacement la consommation électrique de cette adresse au cours de l'année à venir.

Ce notebook s'organise en deux temps. 

- **Premier temps** : Récupération des données et formation d'un data frame regroupant toutes les données intéressantes.   
  Ce premier temps s'organise de la manière suivante :

      1. Récupération des données de consommation électriques individuelles sur le site d'Enedis
      2. Récupération des données météorologiques sur le site InfoClimat à l'échelle de communes, at agrégation de ces données pour constituer des moyennes annuelles
      3. Introduction d'une base de données de l'INSEE pour normaliser les noms de commune de la base de données météorologiques
      4. Constitution d'une liste de communes d'étude
      5. Récupération des données DPE
      6. Fusion des différentes données   

    
- **Second temps** : Régression utiles et conclusions

## Introduction - Mise en route

On commence par installer puis importer toutes les librairies et tous les modules qui nous seront utiles.

In [64]:
!pip install pandas
!pip install geopandas
!pip install geoviews
!pip install lxml
!pip install urllib
!pip install matplotlib
!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a
!pip install mapclassify
!pip install folium

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
  Cloning https://github.com/inseefrlab/cartiflette (to revision 80b8a5a28371feb6df31d55bcc2617948a5f9b1a) to /tmp/pip-req-build-cfd411cc
  Running command git clone --filter=blob:none --quiet https://github.com/inseefrlab/cartiflette /tmp/pip-req-build-cfd411cc
  Running command git rev-parse -q --verify 'sha^80b8a5a28371feb6df31d55bcc2617948a5f9b1a'
  Running command git fetch -q https://github.com/inseefrlab/cartiflette 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Running command git checkout -q 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Resolved https://github.com/inseefrlab/cartiflette to commit 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
!pip install -q lxml

In [15]:
from declarations import *

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=DEPARTEMENT/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 7.17MiB [00:00, 64.8MiB/s]


In [79]:
import bs4
import lxml
import pandas as pd
import urllib
from tqdm.auto import tqdm

from urllib import request

## 1. Construction d'un tableau de données utile à notre étude 

### 1.1. Récupération des données de consommation électrique individuelle

Pour une étude sur la consommation électrique des bâtiments résidentiels, il paraît raisonnable de commencer par importer des donnés de consommation électrique. À cette fin, on utilise la fonction get_data_consumption, codée dans la page declarations.py , pour récupérer les données de consommation par adresse d'Enedis à partir d'URL, et ce pour quatre années différentes.

In [50]:
import declarations as dec

df_cons_2018=dec.get_data_consumption(dec.consumption_data_url_2018, "2018")
df_cons_2019=dec.get_data_consumption(dec.consumption_data_url_2019, "2019")
df_cons_2020=dec.get_data_consumption(dec.consumption_data_url_2020, "2020")
df_cons_2021=dec.get_data_consumption(dec.consumption_data_url_2021, "2021")

Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.
Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


/home/onyxia/work/Projet_Python/declarations.py:108: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


/home/onyxia/work/Projet_Python/declarations.py:108: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


On affiche ensuite l'une des tables pour montrer ce à quoi ressemblent nos données. 

In [70]:
df_cons_2018

,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code INSEE de la commune,Nom de la commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Tri des adresses,Code EPCI,Code Département,Code Région
0,2018,940010000,Ablon-sur-Seine (commune non irisée),18,NaN,ALLEE,DES ORMES,94001,ABLON-SUR-SEINE,RESIDENTIEL,10,25.816,2.582,3.880,18 ALLEE DES ORMES,400010,200054781.0,94.0,11.0
1,2018,940010000,Ablon-sur-Seine (commune non irisée),40,NaN,RUE,DU BAC,94001,ABLON-SUR-SEINE,RESIDENTIEL,18,81.973,4.554,3.880,40 RUE DU BAC,400017,200054781.0,94.0,11.0
2,2018,940010000,Ablon-sur-Seine (commune non irisée),19,NaN,RUE,DU BAC,94001,ABLON-SUR-SEINE,RESIDENTIEL,40,151.116,3.778,3.880,19 RUE DU BAC,400021,200054781.0,94.0,11.0
3,2018,940010000,Ablon-sur-Seine (commune non irisée),5,NaN,RUE,DU CNL PIERRE BROSSOLETTE,94001,ABLON-SUR-SEINE,RESIDENTIEL,13,71.831,5.525,3.880,5 RUE DU CNL PIERRE BROSSOLETTE,400027,200054781.0,94.0,11.0
4,2018,940010000,Ablon-sur-Seine (commune non irisée),16,NaN,RESIDENCE,DU VAL D ABLON,94001,ABLON-SUR-SEINE,RESIDENTIEL,10,29.950,2.995,3.880,16 RESIDENCE DU VAL D ABLON,400031,200054781.0,94.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391094,2018,743150000,Yvoire (commune non irisée),255,NaN,CHEMIN,DES MOTTES,74315,YVOIRE,RESIDENTIEL,10,77.067,7.707,7.256,255 CHEMIN DES MOTTES,790833,200067551.0,74.0,84.0
391095,2018,033210101,Centre Ville,3,NaN,RUE,DE BELLECROIX,3321,YZEURE,RESIDENTIEL,33,149.557,4.532,4.404,3 RUE DE BELLECROIX,790851,200071140.0,3.0,84.0
391096,2018,683860000,Zimmersheim (commune non irisée),23,NaN,RUE,LOUIS PASTEUR,68386,ZIMMERSHEIM,RESIDENTIEL,18,37.803,2.100,7.276,23 RUE LOUIS PASTEUR,790879,200066009.0,68.0,44.0
391097,2018,577640000,Zoufftgen (commune non irisée),39,NaN,RUE,DE LA GARE,57764,ZOUFFTGEN,RESIDENTIEL,17,22.323,1.313,5.711,39 RUE DE LA GARE,790881,245700695.0,57.0,44.0


### 1.2. Scrapping des données météo communales

Une fois les données de consommation électrique récupérées, nous souhaitons nous concentrer sur des données météorologiques à l'échelle la plus locale qu'il nous est possible d'obtenir. Le site Info Climat propose justement des données météorologiques relevées dans plusieurs de ces stations, à une fréquence quotidienne. Par simplicté, nous supposerons que ces rélevés météorologique portent sur toutes la commune dans laquelle est placée la station. 

Les données que nous récupérons sont données à un pas de temps mensuel. Bien qu'il paraitrait pertinent d'utiliser ce pas de temps plus précis que le pas annuel, cela n'est pas possible dans notre étude, étant donné que nous n'avons accès qu'à des données de consommations individuelles. Pour tout de même tâcher de garder de l'information sur ces données météorologiques, nous cherchons à faire des moyennes annuelles. On peut par exemple penser que ces moyennes peuvent nous faire percevoir une élévation du niveau moyen de la température dans certaines communes, avec le réchauffement climatique, et ce peut donc être cet effet qu'une régression de la consommation électrique en fonction de la température pourrait capter.

Pour ordonner les différentes fonctions qui se rapportent à l'extraction de données météorologiques à l'échelle des communes, on se propose de construire une classe regroupant les différentes fonctions qui nous intéressent.

In [149]:
class Meteo :
    """
    Class permettant de scrapper et d'effectuer des opérations sur les données d'infoclimat concernant la température observée par différentes 
    stations météo entre 2011 et 2018.
    """
    def __init__(self):
        """
        Initialisation de la class contenant :
            self.data_all : dict
                Contient l'ensemble des données par année (initialement vides)
        """
        self.année_début=2018
        self.année_fin=2022
        self.data_all = {
                i : {j : {} for j in range(1,13)} for i in range(self.année_début,self.année_fin)
            }

    def scrap(self):
        """
        Fonction permettant de scrapper les données voulues, i.e. les données d'infoclimat concernant la température observée par différentes 
        stations météo entre 2011 et 2018

            Paramètres:
            ------------
                self.data_all : dict
                    Dictionnaire contenant toutes les données par années
        
            Output:
            ----------
                self.data_pop : dict
                    Dictionnaire contenant toutes les données par années actualisées
            
        """
        for y in self.data_all :
            for m in self.data_all[y] :
                url_temp = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
                raw_text = request.urlopen(url_temp).read()
                page = bs4.BeautifulSoup(raw_text,'lxml')
                tableau = page.find('table', {'id' : 'tableau-releves'})
                rows = tableau.find_all('tr')
        
                df = {
            'ville' : [rows[i].find('a').text for i in range(1,len(rows))],
            'tnn' : [rows[i].find('div').text for i in range(1,len(rows))],
            'tnm' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
            'tmm' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
            'txm' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
            'txx' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
            'rr' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
            "ens" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
            'rafale' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                    }
        
                self.data_all[y][m] = df

    def export(self):
        """
        Fonction intégrée d'export pour permettant de faciliter les manipulations.

        Paramètres:
        ------------
            self.data_pop : dict
                Dictionnaire contenant toutes les données par années
        
        Output:
        ----------
            Aucun
        """
        
        self.df_all = pd.DataFrame(self.data_all)
        self.df_all.to_json('data_base.json')

    def liste_min(self, year):
        intersection=self.data_all[year][1]['ville']
        for m in range(2,13):
            intersection=[x for x in intersection if x in self.data_all[year][m]['ville']]
        return intersection

    def df_all_minimal(self,year) :
        intersection = self.liste_min(year)
        df_all_min={j : {} for j in range(1,13)}
        for m in range (1,13):
            df={'ville':[],'tnn':[],'tnm':[],'tmm':[],'txm':[],'txx':[],'rr':[],"ens":[],'rafale':[]}
            for i in range (len(self.data_all[year][m])):
                if self.data_all[year][m]['ville'][i] in intersection : 
                    df['ville'].append(self.data_all[year][m]['ville'][i])
                    df['tnn'].append(self.data_all[year][m]['tnn'][i])
                    df['tnm'].append(self.data_all[year][m]['tnm'][i])
                    df['tmm'].append(self.data_all[year][m]['tmm'][i])
                    df['txm'].append(self.data_all[year][m]['txm'][i])
                    df['txx'].append(self.data_all[year][m]['txx'][i])
                    df['rr'].append(self.data_all[year][m]['rr'][i])
                    df['ens'].append(self.data_all[year][m]['ens'][i])
                    df['rafale'].append(self.data_all[year][m]['rafale'][i])
            df_all_min[m]=df       
        return df_all_min
        

    def tableau_annuel(self,year):
        """
        Fonction intégrée d'export en data frame des données météo pour une année.
        Il faut que cette année "year" ait été importée, c'est-à-dire qu'elle soit entre self.année_début et self.année_fin

        Paramètres:
        ------------
            self.data_pop : dict
                Dictionnaire contenant toutes les données par années
        
        Output:
        ----------
            Aucun
        """
        data_all_min=self.df_all_minimal(year)
        df={
        'ville' : self.liste_min(year), #Si l'on arrive ici, c'est que les villes enregistrées sont les mêmes pendant toute l'année
        'tnn' : sum([data_all_min[m]['tnn'] for m in range (1,13)])/12,
        'tnm' : sum([data_all_min[m]['tnm'] for m in range (1,13)])/12,
        'tmm' : sum([data_all_min[m]['tmm'] for m in range (1,13)])/12,
        'txm' : sum([data_all_min[m]['txm'] for m in range (1,13)])/12,
        'txx' : sum([data_all_min[m]['txx'] for m in range (1,13)])/12,
        'rr' : sum([data_all_min[m]['rr'] for m in range (1,13)])/12,
        'ens' : sum([data_all_min[m]['ens'] for m in range (1,13)])/12,
        'rafale' : sum([data_all_min[m]['rafale'] for m in range (1,13)])/12}
        return pd.DataFrame(df)
        

    def rename_pratique(self):
        """
        Fonction intégrée permettant de renommer facilement les colonnes par le nom utilisé par infoclimat dans le but de faciliter le traitement.

        Paramètres :
        ------------
            self.df_all : DataFrame
                DataFrame contenant toutes les données météo extraites

        Output :
        --------
            self.df_all : DataFrame
                DataFrame contenant les mêmes données mais dont les colonnes ont été renommées
        """
        
        self.df_all = pd.DataFrame(self.data_all)
        self.df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensoleillement du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

    def rename_esthétique(self):
        """
        Fonction intégrée permettant de renommer facilement les colonnes par leur véritable nom, après les avoir simplifié.

        Paramètres :
        ------------
            self.df_all : DataFrame
                DataFrame contenant toutes les données météo extraites

        Output :
        --------
            self.df_all : DataFrame
                DataFrame contenant les mêmes données mais dont les colonnes ont été renommées
        """
        
        self.df_all = pd.DataFrame(self.data_all)
        self.df_all.rename({'ville' : 'Villes', 
            'tnn' : 'Température minimale extrême du mois',
            'tnm' : 'Moyenne des températures minimales du mois', 
            'tmm' : 'Température moyenne du mois', 
            'txm' : 'Moyenne des températures maximales du mois', 
            'txx' : 'Température maximale extrême du mois', 
            'rr' : 'Cumul de précipitation du mois', 
            'ens' : "Heure d'ensoleillement du mois", 
            'rafale' : 'Rafale maximale du mois'},axis=1,inplace = True)

    

In [150]:
#Importation de la class Meteo
lameteo=Meteo()
#On récupère les données
lameteo.scrap()
#On exporte les données au format data_frame
lameteo.export()
#On les affiche
lameteo.df_all

,2018,2019,2020,2021
1,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
2,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
3,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
4,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
5,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
6,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
7,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
8,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
9,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."
10,"{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'Agde (34)', 'Agd...","{'ville': ['Abbeville (80)', 'ACHERES (78)', '...","{'ville': ['Abbeville (80)', 'Abbeville - cent..."


In [141]:
len(lameteo.liste_min(2021))

1292

In [145]:
df={'poto': []}
df['poto'].append(3)
df

{'poto': [3]}

In [151]:
lameteo.rename_pratique()
lameteo.tableau_annuel(2020)

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [113]:
[len(lameteo.df_all[2018][i]['ville']) for i in range(1,13)]

[836, 838, 825, 830, 834, 826, 820, 829, 828, 831, 877, 878]

In [114]:
[len(lameteo.df_all[2019][i]['ville']) for i in range(1,13)]

[889, 885, 962, 964, 971, 972, 973, 975, 981, 984, 992, 995]

In [115]:
[len(lameteo.df_all[2020][i]['ville']) for i in range(1,13)]

[984, 1437, 1330, 1293, 1321, 1329, 1324, 1330, 1342, 1339, 1346, 1360]

In [116]:
[len(lameteo.df_all[2021][i]['ville']) for i in range(1,13)]

[1366, 1430, 1383, 1401, 1397, 1401, 1404, 1406, 1418, 1433, 1448, 1459]

Comme il s'agit de notre seule source de données météo à l'échelle des communes, ce sont ces données qui nous permettront de définir les villes de France auxquelles nous allons nous intéresser.

### 1.3. Utilisation d'une base intermédiaire pour normaliser les noms de communes

In [17]:
communes = s3.download_vectorfile_url_all(
    values = "metropole",
    crs = 4326,
    borders = "COMMUNE",
    vectorfile_format="topojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

communes["area"] = dep.to_crs(2154).area

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 70.4MiB [00:01, 60.2MiB/s]


In [10]:
communes

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,territoire,geometry,area
0,COMMUNE_0000000009754033,NaN,Connangles,CONNANGLES,43076,Commune simple,137,11,1,43,84,200073419,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((3.61421 45.27206, 3.61377 45.27255, ...",5.774291e+09
1,COMMUNE_0000000009760784,NaN,Vidouze,VIDOUZE,65462,Commune simple,243,13,3,65,76,200072106,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-0.02485 43.43002, -0.02500 43.43012...",7.418552e+09
2,COMMUNE_0000000009742077,NaN,Fouesnant,FOUESNANT,29058,Commune simple,9864,11,4,29,53,242900660,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.97912 47.70392, -3.97897 47...",4.033539e+09
3,COMMUNE_0000000009735245,NaN,Plougrescant,PLOUGRESCANT,22218,Commune simple,1166,27,3,22,53,200065928,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.20273 48.84902, -3.20259 48...",4.720673e+09
4,COMMUNE_0000000009752504,NaN,Montcarra,MONTCARRA,38250,Commune simple,569,24,2,38,84,200068542,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((5.43220 45.61477, 5.43220 45.61438, ...",7.365673e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34821,COMMUNE_0000000009742349,NaN,Brie,BRIE,35041,Commune simple,1006,12,1,35,53,243500634,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-1.58072 47.94312, -1.58057 47.94420...",NaN
34822,COMMUNE_0000000009742038,NaN,Orgères,ORGERES,35208,Commune simple,5152,12,3,35,53,243500139,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-1.64908 47.96488, -1.65011 47.96595...",NaN
34823,COMMUNE_0000000009734653,NaN,Castillon-en-Auge,CASTILLON-EN-AUGE,14141,Commune simple,163,19,3,14,28,200069532,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((0.09085 49.04872, 0.09129 49.04872, ...",NaN
34824,COMMUNE_0000000009734657,NaN,Saint-Pierre-en-Auge,SAINT-PIERRE-EN-AUGE,14654,Commune simple,7329,18,3,14,28,200069532,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((0.10231 49.02482, 0.10246 49.02443, ...",NaN


In [15]:
communes['INSEE_COM'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 34826 entries, 0 to 34825
Series name: INSEE_COM
Non-Null Count  Dtype 
--------------  ----- 
34826 non-null  object
dtypes: object(1)
memory usage: 272.2+ KB


In [16]:
communes.nunique()

id            34826
ID                0
NOM           32596
NOM_M         32537
INSEE_COM     34826
STATUT            5
POPULATION     5786
INSEE_CAN        56
INSEE_ARR         9
INSEE_DEP        96
INSEE_REG        13
SIREN_EPCI     1244
source            1
territoire        1
geometry      34826
area             96
dtype: int64

In [17]:
df_cons_2018.nunique()

Année                                                                 1
Code IRIS                                                         18041
Nom IRIS                                                          15180
Numéro de voie                                                     2150
Indice de répétition                                                 18
Type de voie                                                         97
Libellé de voie                                                   47117
Code INSEE de la commune                                           6161
Nom de la commune                                                  6085
Segment de client                                                     1
Nombre de logements                                                 410
Consommation annuelle totale de l'adresse (MWh)                  130608
Consommation annuelle moyenne par logement de l'adresse (MWh)     10840
Consommation annuelle moyenne de la commune (MWh)               

Une fois les données récupérées on peut faire quelques statistiques descriptives (carte de l'intensité des points de consommation etc).

### 1.4. Création d'une liste de villes d'étude

On garde pour la suite le nom des stations utiles. Ces noms nous permettront de filtrer les communes dont nous devons étudier les DPE.

In [40]:
Villes_météo=lameteo.df_all[2018][1]['Villes']

In [41]:
Villes_météo

['Abbeville (80)',
 'Agde (34)',
 'Agde - Le Grau (34)',
 'Agen - La Garenne (47)',
 "Aigrefeuille-d'Aunis (17)",
 "Aigrefeuille-d'Aunis - Château-d'eau (17)",
 'AILLEVILLERS (70)',
 'Aix-en-Provence (13)',
 'Aizenay (85)',
 "Ajaccio - Campo dell'Oro (2A)",
 'Ajaccio - La Parata (2A)',
 'Albi (81)',
 'Albi-Le Séquestre (81)',
 'Alençon - Valframbert (61)',
 'Alenya - Mas Blanc (66)',
 'Almenêches (61)',
 'Ambérieu (01)',
 'Ambérieu-en-Bugey (01)',
 'Ambès (33)',
 'Amuré (79)',
 'Ancey (21)',
 'Angers-Beaucouzé (49)',
 'Anglars-Saint-Félix (12)',
 'Angliers (17)',
 'Aniane (34)',
 'Annecy-Meythet (74)',
 'ANNECY-MEYTHET (FR)',
 'Annonay (07)',
 'Annot (04)',
 'Antibes - La Garoupe (06)',
 'Aouste-sur-Sye (Saint-Pierre) (26)',
 'Arc-sous-Cicon (25)',
 'Arces (17)',
 'Arcizans-Avant (65)',
 'Argenteuil (95)',
 'Armentières (59)',
 'Arnas (69)',
 'Arthun (42)',
 'Artonne (63)',
 'Arvert (1) (17)',
 'Arvert (2) (17)',
 'Ascain (64)',
 'Asnières-sur-Oise (95)',
 'Atuona (987)',
 'AUBENAS SA 

On voit que les noms de stations ne sont pas très propres : ils incluent les numéros de département, voire incluent des quartiers ou regroupent plusieurs villes. On cherche donc à nettoyer ces noms, pour qu'ils puissent être comparés aux noms de commune donnée par l'API utile pour les DPE. 

On ne touche pour cela qu'à la liste "Villes" du data frame. Nous remplacerons ensuite la base initiale par la base nettoyée, et supprimons les lignes qui ne nous intéressent pas dans le data frame après nettoyage. 

L'API des DPE qui nous intéresse n'identifie une commune que par son code INSEE. Il faut donc que, dans la base de données de chaque station, je réussisse à merger la commune de la station avec une base de données qui comprend les codes communes INSEE. 

In [42]:
#Création d'un Data Frame à partir de la colonne "Villes" du dictionnaire précédent
stations=pd.DataFrame(Villes_météo,columns=['Ville'])
stations

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
831,[MAE] Lycée Parc de Vilgénis - MASSY (91)
832,[MAE] Lycée Polyvalent Chevrollier - ANGERS (49)
833,[MAE] Lycée Roz Glas - QUIMPERLE (29)
834,[MAE] Lycée Saint Exupéry - LA ROCHELLE (17)


In [43]:
#On ne garde que les stations dans les villes et pas celles de stations de "Météo à l'école" (MAE)
liste_communes=stations[~stations['Ville'].str[0].isin(['['])]
liste_communes

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
775,Xertigny (88)
776,Xonrupt-Longemer (88)
777,Xonrupt-Longemer - Refuge du Sotré (88)
778,Yssingeaux (43)


On voit qu'en plus par ville il peut il y avoir plusieurs stations (correspondant à plusieurs quartiers). On va donc essayer de supprimer les parties qui ne dénotent pas de la ville dans les noms de stations.

In [44]:
# Méthode très naïve
def nettoyage(liste):
    '''Cette fonction est censée séparer les numéros de département des villes dans une liste présentée comme la liste liste_communes
    Argument : liste (type : list ; liste de string) 
    Sortie : List (type : list ; liste de listes)'''
    List=[]
    for ville in liste:
        i=0
        mot=''
        while i < len(ville) and ville[i+1] != '(' :
            mot+=ville[i]
            i+=1
        mot=mot.upper()
        List.append([mot,ville[-3:-1]])
    
    return List
List=nettoyage(liste_communes['Ville'])
List

[['ABBEVILLE', '80'],
 ['AGDE', '34'],
 ['AGDE - LE GRAU', '34'],
 ['AGEN - LA GARENNE', '47'],
 ["AIGREFEUILLE-D'AUNIS", '17'],
 ["AIGREFEUILLE-D'AUNIS - CHÂTEAU-D'EAU", '17'],
 ['AILLEVILLERS', '70'],
 ['AIX-EN-PROVENCE', '13'],
 ['AIZENAY', '85'],
 ["AJACCIO - CAMPO DELL'ORO", '2A'],
 ['AJACCIO - LA PARATA', '2A'],
 ['ALBI', '81'],
 ['ALBI-LE SÉQUESTRE', '81'],
 ['ALENÇON - VALFRAMBERT', '61'],
 ['ALENYA - MAS BLANC', '66'],
 ['ALMENÊCHES', '61'],
 ['AMBÉRIEU', '01'],
 ['AMBÉRIEU-EN-BUGEY', '01'],
 ['AMBÈS', '33'],
 ['AMURÉ', '79'],
 ['ANCEY', '21'],
 ['ANGERS-BEAUCOUZÉ', '49'],
 ['ANGLARS-SAINT-FÉLIX', '12'],
 ['ANGLIERS', '17'],
 ['ANIANE', '34'],
 ['ANNECY-MEYTHET', '74'],
 ['ANNECY-MEYTHET', 'FR'],
 ['ANNONAY', '07'],
 ['ANNOT', '04'],
 ['ANTIBES - LA GAROUPE', '06'],
 ['AOUSTE-SUR-SYE', '26'],
 ['ARC-SOUS-CICON', '25'],
 ['ARCES', '17'],
 ['ARCIZANS-AVANT', '65'],
 ['ARGENTEUIL', '95'],
 ['ARMENTIÈRES', '59'],
 ['ARNAS', '69'],
 ['ARTHUN', '42'],
 ['ARTONNE', '63'],
 ['ARVERT',

In [45]:
def nettoyage_bis(liste):
    '''Cette fonction est censée séparer les numéros de département des villes dans une liste présentée comme la liste liste_communes
    Argument : liste (type : list ; liste de string) 
    Sortie : List (type : list ; liste de listes)'''
    List=[]
    for ville in liste:
        i=0
        mot=''
        while i < len(ville) and ville[i+1] != '(' :
            mot+=ville[i]
            i+=1
        mot=mot.upper()
        List.append(mot)
    
    return List
List_2=nettoyage_bis(liste_communes['Ville'])
dp_list=pd.DataFrame(List_2,columns=['Ville'])
dp_list

,Ville
0,ABBEVILLE
1,AGDE
2,AGDE - LE GRAU
3,AGEN - LA GARENNE
4,AIGREFEUILLE-D'AUNIS
...,...
775,XERTIGNY
776,XONRUPT-LONGEMER
777,XONRUPT-LONGEMER - REFUGE DU SOTRÉ
778,YSSINGEAUX


On cherche à savoir combien de villes on est amenés à garder si on ne veut pas trop se casser la tête et ne garder que les noms "jolis" de stations, pour lesquels on peut facilement trouver le nom de la ville d'origine.

In [52]:
filtre=dp_list.merge(communes, left_on='Ville', right_on='NOM_M')[['NOM_M','STATUT','POPULATION','INSEE_COM','geometry']]
filtre

,NOM_M,STATUT,POPULATION,INSEE_COM,geometry
0,ABBEVILLE,Sous-préfecture,22980,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ..."
1,AGDE,Commune simple,29600,34003,"MULTIPOLYGON (((3.50101 43.26349, 3.50130 43.2..."
2,AIGREFEUILLE-D'AUNIS,Commune simple,4143,17003,"POLYGON ((-0.98001 46.10352, -0.98001 46.10372..."
3,AIX-EN-PROVENCE,Sous-préfecture,145133,13001,"POLYGON ((5.50630 43.53216, 5.50615 43.53177, ..."
4,AIZENAY,Commune simple,9881,85003,"POLYGON ((-1.54264 46.72417, -1.54249 46.72358..."
...,...,...,...,...,...
396,WINTERSBOURG,Commune simple,269,57747,"POLYGON ((7.19741 48.77898, 7.19668 48.77917, ..."
397,XERTIGNY,Commune simple,2586,88530,"POLYGON ((6.45837 48.04351, 6.45807 48.04126, ..."
398,XONRUPT-LONGEMER,Commune simple,1515,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ..."
399,YSSINGEAUX,Sous-préfecture,7278,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ..."


In [56]:
filtre_conso_2018 = filtre.merge(df_cons_2018, left_on="NOM_M", right_on="Nom de la commune")[['NOM_M','STATUT','INSEE_COM','geometry','Code IRIS', 'Nom IRIS', 'Nombre de logements', "Consommation annuelle totale de l'adresse (MWh)", "Consommation annuelle moyenne par logement de l'adresse (MWh)", "Consommation annuelle moyenne de la commune (MWh)", "Adresse"]]   
filtre_conso_2018

,NOM_M,STATUT,INSEE_COM,geometry,Code IRIS,Nom IRIS,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse
0,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010501,La Porte au Bois,28,134.006,4.786,3.522,39 ROUTE D AMIENS
1,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,36,91.020,2.528,3.522,10 RUE DE GASCOGNE
2,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010503,Ponthieu-Bagatelle,10,18.327,1.833,3.522,5 RUE DE GUYANE
3,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,10,37.592,3.759,3.522,15 RUE DE NORMANDIE
4,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,10,33.586,3.359,3.522,13 RUE DE PICARDIE
...,...,...,...,...,...,...,...,...,...,...,...
25167,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,10,12.318,1.232,3.968,10 AVENUE DU MARECHAL DE VAUX
25168,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,12,10.403,0.867,3.968,165 RUE LOUIS JOUVET
25169,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,12,11.531,0.961,3.968,85 RUE LOUIS JOUVET
25170,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,18,17.056,0.948,3.968,200 RUE SAINT ROCH


In [78]:
filtre_conso_2018['Adresse normalisée']=filtre_conso_2018['Adresse']+ ' '+ filtre_conso_2018['INSEE_COM']+ ' '+ filtre_conso_2018['NOM_M']
filtre_conso_2018

,NOM_M,STATUT,INSEE_COM,geometry,Code IRIS,Nom IRIS,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Adresse normalisée
0,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010501,La Porte au Bois,28,134.006,4.786,3.522,39 ROUTE D AMIENS,39 ROUTE D AMIENS 80001 ABBEVILLE
1,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,36,91.020,2.528,3.522,10 RUE DE GASCOGNE,10 RUE DE GASCOGNE 80001 ABBEVILLE
2,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010503,Ponthieu-Bagatelle,10,18.327,1.833,3.522,5 RUE DE GUYANE,5 RUE DE GUYANE 80001 ABBEVILLE
3,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,10,37.592,3.759,3.522,15 RUE DE NORMANDIE,15 RUE DE NORMANDIE 80001 ABBEVILLE
4,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,10,33.586,3.359,3.522,13 RUE DE PICARDIE,13 RUE DE PICARDIE 80001 ABBEVILLE
...,...,...,...,...,...,...,...,...,...,...,...,...
25167,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,10,12.318,1.232,3.968,10 AVENUE DU MARECHAL DE VAUX,10 AVENUE DU MARECHAL DE VAUX 43268 YSSINGEAUX
25168,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,12,10.403,0.867,3.968,165 RUE LOUIS JOUVET,165 RUE LOUIS JOUVET 43268 YSSINGEAUX
25169,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,12,11.531,0.961,3.968,85 RUE LOUIS JOUVET,85 RUE LOUIS JOUVET 43268 YSSINGEAUX
25170,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,18,17.056,0.948,3.968,200 RUE SAINT ROCH,200 RUE SAINT ROCH 43268 YSSINGEAUX


### 1.5. Récupération des données de Diagnostics de Performance Energétique (DPE)

On commence par programmer une fonction qui permet de récupérer des données à partir d'une APL. Voici l'exemple de Lino lorsqu'on essaye la demande avec une seule ville.

In [7]:
code_commune = "01450"
size = 100
api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
url_api = f"{api_root}?page=1&after=10&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=neighbors"

In [70]:
import requests
import pandas as pd
import geopandas as gpd

def get_dpe_from_url(api_url):
    '''Argument:
        api_url (string) : url de l'API voulue
    Sortie:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

dpe = get_dpe_from_url(url_api)
dpe

,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,classe_estimation_ges,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,version_methode_dpe,geometry
0,D,Vente,1947,"45.925922,5.229964",45.925922,117.16,487,Maison Individuelle,Rue de la Brugnière 01800 Villieu-Loyes-Mollon,23215,...,E,Méthode Facture,01,178.00,2013-06-13,5.229964,None,04JZNel3WCJYcfsHpCcHv,NaN,POINT (5.22996 45.92592)
2,D,Neuf,2006,"45.923421,5.223777",45.923421,90.53,689,Maison Individuelle,Chemin du Pont-vieux 01800 Villieu-Loyes-Mollon,401672,...,C,FACTURE - DPE,01,227.99,2013-06-11,5.223777,None,rkdV2lJn2wxaidVBaHBFY,V2012,POINT (5.22378 45.92342)
3,D,Neuf,2006,"45.923068,5.223271",45.923068,105.27,690,Maison Individuelle,Lotissement le Pont Vieux 01800 Villieu-Loyes-...,428929,...,C,FACTURE - DPE,01,216.38,2013-06-11,5.223271,None,P92FDkFFgWqoktiPEptkC,V2012,POINT (5.22327 45.92307)
6,E,Vente,1983,"45.91956,5.224214",45.919560,130.00,1127,Maison Individuelle,Rue du Pollet 01800 Villieu-Loyes-Mollon,848537,...,C,Méthode 3CL,01,309.00,2013-07-09,5.224214,None,jBNxOg77oHrmTW6EIyTR5,"3CL-DPE, version 1.3",POINT (5.22421 45.91956)
7,E,Vente,2006,"45.916251,5.221846",45.916251,90.34,1374,Maison Individuelle,6 Rue des Terres du Pollet 01800 Villieu-Loyes...,598270,...,C,3CL - DPE,01,284.00,2013-07-16,5.221846,None,MuAKfgQyjAM0GHTLgErrm,2012,POINT (5.22185 45.91625)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C,Location,2019,"45.920228,5.222557",45.920228,54.82,102717,Logement,25 Rue du Bottet 01800 Villieu-Loyes-Mollon,235935,...,D,3CL - DPE,01,94.36,2020-10-21,5.222557,None,bOsqqc-zpzRKA7u_8Pdj7,V2012,POINT (5.22256 45.92023)
96,B,Location,2019,"45.920228,5.222557",45.920228,65.89,102718,Logement,25 Rue du Bottet 01800 Villieu-Loyes-Mollon,140973,...,C,3CL - DPE,01,74.80,2020-10-21,5.222557,None,rgH9CYqZTZuJCxurcUaub,V2012,POINT (5.22256 45.92023)
97,B,Location,2019,"45.920292,5.223036",45.920292,67.77,102719,Logement,257 Avenue Charles de Gaulle 01800 Villieu-Loy...,912454,...,C,3CL - DPE,01,65.00,2020-10-21,5.223036,None,l1yvcYwWAV9UQtnTg48nb,V2012,POINT (5.22304 45.92029)
98,C,Location,2019,"45.920292,5.223036",45.920292,58.23,102720,Logement,257 Avenue Charles de Gaulle 01800 Villieu-Loy...,755526,...,D,3CL - DPE,01,102.29,2020-10-21,5.223036,None,k4E_1qA9XKd6Dps8ceBZf,V2012,POINT (5.22304 45.92029)


In [69]:
list(dpe)

['classe_consommation_energie',
 'tr001_modele_dpe_type_libelle',
 'annee_construction',
 '_geopoint',
 'latitude',
 'surface_thermique_lot',
 '_i',
 'tr002_type_batiment_description',
 'geo_adresse',
 '_rand',
 'code_insee_commune_actualise',
 'estimation_ges',
 'geo_score',
 'classe_estimation_ges',
 'nom_methode_dpe',
 'tv016_departement_code',
 'consommation_energie',
 'date_etablissement_dpe',
 'longitude',
 '_score',
 '_id',
 'version_methode_dpe',
 'geometry']

Il est trop coûteux de récupérer l'intégralité des données de DPE, et nous ne disposons de données météo que pour certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de les agréger dans une unique table de données.

In [72]:
liste_code_commune = filtre['INSEE_COM']
def gros_data_frame(liste_commune):
    DPE=pd.DataFrame(columns=[list(dpe)])
    for i in liste_commune: 
        code_commune=i
        size = 100
        api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
        url_api = f"{api_root}?page=1&after=10&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=neighbors"
        DPE=pd.concat([DPE,get_dpe_from_url(url_api)])


big_data=gros_data_frame(liste_code_commune)
big_data

AttributeError: 'DataFrame' object has no attribute 'longitude'

### 1.6 Fusion des tableaux

Il ne nous reste enfin plus qu'à fusionner les tableaux pour obtenir dans une même ligne toutes les données qui nous semblent importantes, à savoir les données de consommations, les données DPE et les données météorologiques annuelles.

In [ ]:
for mot in big_data['geo_adresse'] :
    mot.upper()

data_dpe_2018=filtre_conso_2018.merge(big_data, left_on='Adresse normalisée', right_on='geo_adresse')
data_dpe_2018

In [ ]:
data_reg_2018=data_dpe_2018.merge(meteo_2018, left_on='NOM_M', right_on='Villes')